In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5,), (0.5,0.5,0.5,))]
)

trainset = torchvision.datasets.CIFAR(train=True, download=True, transforms=transform)
testset = torchvision.datasets.CIFAR(